# Tutorial 2
We show how to train DeepScalper on BTC trading.

## Step 1: Import Packages
Modify the system path and load the corresponding packages and functions 

In [ ]:
import os
import sys

import warnings
warnings.filterwarnings("ignore")

ROOT = os.path.dirname(os.path.abspath("."))
sys.path.append(ROOT)

import torch
import argparse
import os.path as osp
from mmcv import Config
from trademaster.utils import replace_cfg_vals
from trademaster.nets.builder import build_net
from trademaster.environments.builder import build_environment
from trademaster.datasets.builder import build_dataset
from trademaster.agents.builder import build_agent
from trademaster.optimizers.builder import build_optimizer
from trademaster.losses.builder import build_loss
from trademaster.trainers.builder import build_trainer
from trademaster.transition.builder import build_transition
from trademaster.utils import plot
from trademaster.utils import set_seed
set_seed(2023)

In [ ]:
#read the 5m_data.csv file  
import pandas as pd

csv = pd.read_csv('/home/krow/Desktop/coding/loltrain/deepscalp/TradeMaster/data/algorithmic_trading/BTC2/5m_data.csv')
csv

In [ ]:
#rename date to date
csv.rename(columns={'Date':'date'}, inplace=True)
csv

In [ ]:
#rename last_price to close
csv.rename(columns={'last_price':'close'}, inplace=True)
csv

In [ ]:
#split into train, validate and test and save as csv in same folder
train = csv[:int(0.7*len(csv))]
validate = csv[int(0.7*len(csv)):int(0.85*len(csv))]
test = csv[int(0.85*len(csv)):]
train.to_csv('/home/krow/Desktop/coding/loltrain/deepscalp/TradeMaster/data/algorithmic_trading/BTC2/train.csv')
validate.to_csv('/home/krow/Desktop/coding/loltrain/deepscalp/TradeMaster/data/algorithmic_trading/BTC2/valid.csv')
test.to_csv('/home/krow/Desktop/coding/loltrain/deepscalp/TradeMaster/data/algorithmic_trading/BTC2/test.csv')

## Step 2: Load Configs
Load default config from the file `configs/algorithmic_trading/algorithmic_trading_BTC_deepscalper_deepscalper_adam_mse.py`

In [ ]:

parser = argparse.ArgumentParser(description='Open s1m data')
parser.add_argument("--config", default=osp.join(ROOT, "configs", "algorithmic_trading", "deepsn1per.py"),
                    help="download datasets config file path")
parser.add_argument("--task_name", type=str, default="train")
args, _= parser.parse_known_args()

cfg = Config.fromfile(args.config)
task_name = args.task_name
cfg = replace_cfg_vals(cfg)

## Step 3: Build Dataset

In [ ]:
dataset = build_dataset(cfg)

In [ ]:
train_environment = build_environment(cfg, default_args=dict(dataset=dataset, task="train"))
valid_environment = build_environment(cfg, default_args=dict(dataset=dataset, task="valid"))
test_environment = build_environment(cfg, default_args=dict(dataset=dataset, task="test"))
train_environment.df.head()

## Step 5: Build Net 
Update information about the state and action dimension. Create networks and optimizer for DeepScalper


In [ ]:
action_dim = train_environment.action_dim
state_dim = train_environment.state_dim

cfg.act.update(dict(action_dim=action_dim, state_dim=state_dim))
act = build_net(cfg.act)
act_optimizer = build_optimizer(cfg, default_args=dict(params=act.parameters()))
if cfg.cri:
    cfg.cri.update(dict(action_dim=action_dim, state_dim=state_dim))
    cri = build_net(cfg.cri)
    cri_optimizer = build_optimizer(cfg, default_args=dict(params=cri.parameters()))
else:
    cri = None
    cri_optimizer = None

## Step 6: Build Loss

In [ ]:
criterion = build_loss(cfg)

## Step 7: Build Transition

In [ ]:
transition = build_transition(cfg)


## Step 8: Build Agent

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
agent = build_agent(cfg, default_args=dict(action_dim = action_dim,state_dim = state_dim,act = act,cri = cri,act_optimizer = act_optimizer,cri_optimizer = cri_optimizer, criterion = criterion,transition = transition,device=device))

## Step 9: Build Trainer
Build trainer from config and create work directionary to save the result, model and config

In [ ]:
trainer = build_trainer(cfg, default_args=dict(train_environment=train_environment,valid_environment=valid_environment,test_environment=test_environment,agent=agent,device=device))

## Step 10: Train the RL Agent
Train the trainer based on the config and save results in workdir

In [ ]:
trainer.train_and_valid()

## Step 11: RL Agent Testing

In [ ]:
trainer.test()

In [ ]:
plot(trainer.test_environment.save_asset_memory(),alg="DeepScalper")

In [ ]:
# Initialize the model architecture as during training
act = build_net(cfg.act)
if cfg.cri:
    cri = build_net(cfg.cri)
else:
    cri = None

In [ ]:
import torch

checkpoint = torch.load('/home/krow/Desktop/coding/loltrain/deepscalp/TradeMaster/work_dir/algorithmic_trading_BTC2_deepscalper_deepscalper_adam_mse/checkpoints/best.pth')

# Assuming the checkpoint contains model states under keys like 'act', 'cri', you would extract them like this:
act_state_dict = checkpoint['act']
cri_state_dict = checkpoint['cri']  # If you have a critic model

# Then, load these state dictionaries into your models
act.load_state_dict(act_state_dict)
if cri is not None:
    cri.load_state_dict(cri_state_dict)
# Move models to the appropriate device (CPU/GPU)
act.to('cuda')
if cri is not None:
    cri.to('cuda')

# Ensure the models are in evaluation mode
act.eval()
if cri is not None:
    cri.eval()

In [ ]:
import pandas as pd 
test_df = pd.read_csv('/home/krow/Desktop/coding/loltrain/deepscalp/TradeMaster/data/algorithmic_trading/BTC2/test.csv')
test_df

In [ ]:
#convert date to float
test_df['date'] = pd.to_datetime(test_df['date'])
test_df['date'] = pd.to_numeric(test_df['date'])
#drop unnamed
test_df = test_df.drop('Unnamed: 0', axis=1)
test_df = test_df.tail(64)
test_df


In [ ]:
import numpy as np

numpy_array = test_df.to_numpy()

tensor = torch.from_numpy(numpy_array)

tensor = tensor.to('cuda')



In [ ]:
from trademaster.agents.algorithmic_trading import AlgorithmicTradingDQN
model = agent
PATH_TO_SAVED_MODEL = '/home/krow/Desktop/coding/loltrain/deepscalp/TradeMaster/work_dir/algorithmic_trading_BTC2_deepscalper_deepscalper_adam_mse/checkpoints/best.pth'
model.load_state_dict(torch.load(PATH_TO_SAVED_MODEL))


In [ ]:
with torch.no_grad():
    action = model.predict(tensor)
    print(action)